In [ ]:
from ipywidgets import interactive
import getpass, requests, os, pathlib, shutil
# Standard modules to use and manipulate dataframes
import pandas as pd, numpy as np

In [ ]:
#General
hname = "osquery.io"
location = %pwd 
#Windows
twlocation = location + "\\table_data_win"
pw = pathlib.Path(twlocation)
#extra "" on purpose because it needs to pass this argument as-is to commandline
osquery_w_location = '"C:\Program Files\osquery\osqueryi.exe"'
#Linux
#maybe https://gist.github.com/parente/b6ee0efe141822dfa18b6feeda0a45e5 ??
osquery_l_location = !wsl -d Ubuntu which osqueryi
osquery_l_location = ''.join(osquery_l_location)
tllocation = location + "/table_data_lnx"
pl = pathlib.Path(tllocation)
lnx_osq = ''
#WSL
twsllocation = location + "\\table_data_wsl"
pwsl = pathlib.Path(twsllocation)
VM_Name = ''
wsl_osq = 'wsl -d'+' '+VM_Name+' '
win_osq = '"C:\Program Files\osquery\\"'


In [ ]:
def installed_distro():
#maybe https://gist.github.com/parente/b6ee0efe141822dfa18b6feeda0a45e5 ??
    #Create a list of installed Linux distributions
    distro_lst = !wsl --list -q
    #Remove NULL and empty strings from the list
    distro_lst = [d.replace('\x00', '') for d in distro_lst]
    distro_lst = list(filter(None, distro_lst))
    return(distro_lst)

def get_installed(Distro):
    #Just a small function to be used to return the distibution chosen in the `interactive`
    return(Distro)

distro_dd = interactive(get_installed, Distro=installed_distro())

In [ ]:
def create_folder(p,tlocation):
    #Create folder for the tables
    try:
        p.mkdir()
    except:
        print ("Creation of the directory %s failed, location probably already exists" % tlocation)
    else:
        print ("Successfully created the directory %s " % tlocation)

In [ ]:
def install_on_linux(os_osq):
    !{os_osq} sudo apt-get update
    !{os_osq} sudo apt-get -y install gnupg software-properties-common
    OSQUERY_KEY='1484120AC4E9F8A1A577AEEE97A80C63C9D8B80B'
    !{os_osq} sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv-keys {OSQUERY_KEY}
    !{os_osq} sudo add-apt-repository 'deb [arch=amd64] https://pkg.osquery.io/deb deb main'
    !{os_osq} sudo apt-get update
    !{os_osq} sudo apt-get -y install osquery

In [ ]:
#For using sudo with password, example
#!{os_osq}echo {password} ^| sudo -S apt-get update
#!{os_osq}echo {password} ^| sudo -S apt-get -y install osquery

In [ ]:
def install_osquery_wsl_linux(os_osq):
    if "wsl" in os_osq:
        os_osq = 'wsl -d '+VM_Name+' '
        osq_command = !{os_osq}which osqueryi
        if not osq_command.spstr:
            install_on_linux(os_osq)
        else:
            !{os_osq}osqueryi --version
            print("Osquery already installed")
    elif shutil.which("osqueryi") is None:
        install_on_linux(os_osq)
    else:
        !{os_osq}osqueryi --version
        print("Osquery already installed")

In [ ]:
def install_osquery_windows(win_osq):
    try:
        !{win_osq}osqueryi --version
        print("Osquery already installed")
    except:
        version = "4.3.0"
        url = f'https://pkg.osquery.io/windows/osquery-{version}.msi'
        #First find name of file to be used for installing
        if url.find('/'):
            osquery_filename = url.rsplit('/', 1)[1]
            #Download file to current location
            r = requests.get(url, allow_redirects=True)
            open(osquery_filename, 'wb').write(r.content)
        #Install the Osquery MSI without interaction and also write out a log file in the same location to check if anything went wrong.
        !msiexec /i $osquery_filename /quiet /log "osquery-install.log"

In [ ]:
def osquery_tables_os(os_osq):
    if "wsl" in os_osq:
        os_osq = 'wsl -d'+' '+VM_Name+' '
        osq_tables = !{os_osq}osqueryi .tables
        osq_tables = osq_tables.fields(1)
    else:
        #pass
        osq_tables = !{os_osq}osqueryi .tables
        osq_tables = osq_tables.fields(1)
    return(osq_tables)

In [ ]:
osquery_tables = ['osquery_events','osquery_extensions','osquery_flags','osquery_info','osquery_packs','osquery_registry','osquery_schedule']
curl_tables = ['curl','curl_certificate']

where_tables_l = ['elf_dynamic','elf_info','elf_sections','elf_segments','elf_symbols','file','hash','magic','yara']
#Tables that are event tables or tables that have not uniform WHERE constraints
error_tables_l = ['file_events','hardware_events','process_events','process_file_events','selinux_events','socket_events','syslog_events','user_events','yara_events','carves','device_file','device_partitions','docker_container_fs_changes','docker_container_processes','docker_container_stats','lxd_instance_config','lxd_instance_devices','prometheus_metrics','device_hash','lldp_neighbors','smart_drive_info','carbon_black_info','augeas']

where_tables_w = ['authenticode','file','hash','ntfs_acl_permissions']
#Tables that are event tables or tables that have not uniform WHERE constraints
error_tables_w = ['ntfs_journal_events','powershell_events','windows_events','carves','carbon_black_info','chocolatey_packages']

In [ ]:
def run_on_wsl_linux(os_osqr,tables,tlocation):
    for wt in tables:
        if wt not in error_tables_l and wt not in osquery_tables:
            flocation = tlocation + wt 
            if wt in curl_tables:
                if wt == 'curl':
                    command = '"SELECT * from '+wt+' WHERE url = \'https://'+hname+'\';"'
                    if "wsl" in os_osqr:
                        !{os_osqr} sudo osqueryi {command} --json > {flocation}.json
                    else:
                        !$os_osqr sudo osqueryi $command --json > $flocation'.json'
                else:
                    command = '"SELECT * from '+wt+' WHERE hostname = \''+hname+'\';"'
                    if "wsl" in os_osqr:
                        !{os_osqr} sudo osqueryi {command} --json > {flocation}.json
                    else:
                        !$os_osqr sudo osqueryi $command --json > $flocation'.json'
            elif wt in where_tables_l:
                command = '"SELECT * from '+wt+' WHERE path = \''+osquery_l_location+'\';"'
                if "wsl" in os_osqr:
                    !{os_osqr} sudo osqueryi {command} --json > {flocation}.json
                else:
                    !$os_osqr sudo osqueryi $command --json > $flocation'.json'
            else:
                command = '"SELECT * from '+wt+';"'
                if "wsl" in os_osqr:
                    !{os_osqr} sudo osqueryi {command} --json > {flocation}.json
                else:
                    !$os_osqr sudo osqueryi $command --json > $flocation'.json'

In [ ]:
def run_osquery_wsl_linux(os_osq,p,tables,tlocation):#,password):
    create_folder(p,tlocation)
    #limit = 50
    if "wsl" in os_osq:
        os_osqr = 'wsl -d'+' '+VM_Name+' '
        flocation = tlocation + '\\' 
        run_on_wsl_linux(os_osqr,tables,flocation)
    else:
        os_osqr = os_osq
        flocation = tlocation + '/' 
        run_on_wsl_linux(os_osqr,tables,flocation)

In [ ]:
def run_osquery_windows(p,tables,tlocation):
    create_folder(p,tlocation)
    osquery_w_location = '"C:\Program Files\osquery\osqueryi.exe"'
    #used specifically for the queries, otherwise issues with '' and ""
    osquery_bin = "C:\Program Files\osquery\osqueryi.exe"
    limit = 50
    for wt in tables:
        if wt not in error_tables_w and wt not in osquery_tables:
            flocation = tlocation + '\\' + wt
            if wt in curl_tables:
                if wt == 'curl':
                    !{osquery_w_location} "SELECT * from {wt} WHERE url = 'https://{hname}';" --json > {flocation}.json
                else:
                    !{osquery_w_location} "SELECT * from {wt} WHERE hostname = '{hname}';" --json > {flocation}.json
            elif wt in where_tables_w:
                !{osquery_w_location} "SELECT * from {wt} WHERE path = '{osquery_bin}';" --json > {flocation}.json #LIMIT {limit}
            else:
                !{osquery_w_location} "SELECT * from {wt};" --json > {flocation}.json #LIMIT {limit}
        else:
            pass

In [ ]:
def osquery_data_extract(p):
    data_columns = []
    for path in p.rglob("*.json"):
        os_t = path.stem
        data = pd.read_json(path, orient='records', encoding='ANSI')
        data = data.add_prefix(os_t+'.')
        for i in range(0, data.shape[0]):
            column_data = list(data.columns.values)
            value_data = list(data.values[i])
            column_value_data = list(zip(column_data,value_data))
            data_columns.append(column_value_data)
    return(data_columns)

In [ ]:
def filter_osquery_data(df):
    #Split Table.Column into separate columns
    temp_table = df['Table.Column'].str.split(".", n = 1, expand = True)
    df['Table'] = temp_table[0]
    df['Column'] = temp_table[1]
    df = df[['Table','Table.Column','Column','Data']]

    #Remove null and empty data fields
    extract_df_clean = df[(df['Data'].notnull()) & (df['Data'] != '')]
    extract_df_clean.drop_duplicates(keep='first', inplace=True) 

    #Filter out all entries with 2 or less characters in the Data field, these entries are too general to make good joins
    extract_df_clean_filter = extract_df_clean[extract_df_clean.Data.map(str).apply(len) > 2]
    return(extract_df_clean_filter)

In [ ]:
def anonymize_data(df):
    #Anonimze Data column, create new column for now, drop Data later
    extract_df_clean_full = df
    extract_df_clean_full['anon'] = pd.factorize(extract_df_clean_full.Data)[0]
    extract_df_clean_full = extract_df_clean_full[extract_df_clean_full.anon != -1]
    #Drop data to anonymize the table completely
    extract_df_clean_full.drop(columns=['Data'],inplace=True)
    return(extract_df_clean_full)

In [ ]:
def get_dup_data(df):
    extract_df_clean_filter_dup = df[df.duplicated(subset='Data', keep=False)]
    extract_df_clean_filter_dup_anon = anonymize_data(extract_df_clean_filter_dup)
    return(extract_df_clean_filter_dup_anon)